In [1]:
%matplotlib inline
%run ../../import_envs.py
print('probtorch:', probtorch.__version__, 
      'torch:', torch.__version__, 
      'cuda:', torch.cuda.is_available())

probtorch: 0.0+5a2c637 torch: 0.4.1 cuda: True


In [2]:
## Load dataset
data_path = "../../../rings_fixed_radius"
Data = torch.from_numpy(np.load(data_path + '/obs.npy')).float()
FIXED_RADIUS = 1.5

NUM_DATASETS, N, D = Data.shape
K = 5 ## number of clusters
SAMPLE_SIZE = 10
NUM_HIDDEN_GLOBAL = 128
NUM_HIDDEN_LOCAL = 64
NUM_STATS = 64

MCMC_SIZE = 10
BATCH_SIZE = 20
NUM_EPOCHS = 100
LEARNING_RATE = 5 * 1e-4

CUDA = torch.cuda.is_available()
PATH = 'soft-5rings-%dsteps-%dsamples' % (MCMC_SIZE, SAMPLE_SIZE)
DEVICE = torch.device('cuda:1')

obs_rad = torch.ones(1) * FIXED_RADIUS
noise_sigma = torch.ones(1) * 0.05
if CUDA:
    obs_rad = obs_rad.cuda().to(DEVICE)
    noise_sigma = noise_sigma.cuda().to(DEVICE)
Train_Params = (NUM_EPOCHS, NUM_DATASETS, SAMPLE_SIZE, BATCH_SIZE, CUDA, DEVICE, PATH)
Model_Params = (obs_rad, noise_sigma, N, K, D, MCMC_SIZE)

In [3]:
from local_enc_mu import *
from global_oneshot_mu_v2 import *
from global_enc_mu_v2 import *
## if reparameterize continuous variables
Reparameterized = False
# initialization
enc_z = Enc_z(K, D, NUM_HIDDEN_LOCAL, CUDA, DEVICE)
enc_mu = Enc_mu(K, D, NUM_HIDDEN_GLOBAL, NUM_STATS, CUDA, DEVICE, Reparameterized)
oneshot_mu = Oneshot_mu(K, D, NUM_HIDDEN_GLOBAL, NUM_STATS, CUDA, DEVICE, Reparameterized)
if CUDA:
    enc_z.cuda().to(DEVICE)
    enc_mu.cuda().to(DEVICE)
    oneshot_mu.cuda().to(DEVICE)
optimizer =  torch.optim.Adam(list(oneshot_mu.parameters())+list(enc_mu.parameters())+list(enc_z.parameters()),lr=LEARNING_RATE, betas=(0.9, 0.99))
models = (oneshot_mu, enc_mu, enc_z)

In [4]:
# enc_z.load_state_dict(torch.load("../weights/enc-z-%s" % PATH))
# enc_mu.load_state_dict(torch.load("../weights/enc-mu-%s" % PATH))
# oneshot_mu.load_state_dict(torch.load("../weights/oneshot-mu-%s" % PATH))

In [ ]:
from ag_ep import *
train(models, EUBO_fixed_radi, optimizer, Data, Model_Params, Train_Params)

epoch: 0\100 (205s),  symKL_DB_eta: 174293.544,  symKL_DB_z: 837953.084,  gap_eta: 174189.445,  gap_z: 837725.119,  loss: -112545.520,  ess: 2.669
epoch: 1\100 (206s),  symKL_DB_eta: 45749.752,  symKL_DB_z: 82385.582,  gap_eta: 45714.251,  gap_z: 82241.010,  loss: -41350.094,  ess: 4.158
epoch: 2\100 (207s),  symKL_DB_eta: 17280.132,  symKL_DB_z: 15578.319,  gap_eta: 17270.907,  gap_z: 15476.452,  loss: -26646.518,  ess: 4.573
epoch: 3\100 (206s),  symKL_DB_eta: 11342.409,  symKL_DB_z: 8251.699,  gap_eta: 11335.425,  gap_z: 8166.728,  loss: -20051.505,  ess: 4.676
epoch: 4\100 (205s),  symKL_DB_eta: 8651.696,  symKL_DB_z: 6768.798,  gap_eta: 8647.467,  gap_z: 6692.402,  loss: -16690.096,  ess: 4.712
epoch: 5\100 (205s),  symKL_DB_eta: 6968.922,  symKL_DB_z: 6140.915,  gap_eta: 6952.200,  gap_z: 6063.610,  loss: -14480.888,  ess: 4.730
epoch: 6\100 (205s),  symKL_DB_eta: 5776.094,  symKL_DB_z: 5780.427,  gap_eta: 5759.877,  gap_z: 5690.884,  loss: -12838.696,  ess: 4.741
epoch: 7\100 (2

In [ ]:
# torch.save(enc_z.state_dict(), "../weights/enc-z-%s" % PATH)
# torch.save(enc_mu.state_dict(), "../weights/enc-mu-%s" % PATH)
# torch.save(oneshot_mu.state_dict(), "../weights/oneshot-mu-%s" % PATH)

In [ ]:
from ag_ep import *
BATCH_SIZE_TEST = 50
Train_Params_Test = (NUM_EPOCHS, NUM_DATASETS, SAMPLE_SIZE, BATCH_SIZE_TEST, CUDA, DEVICE, PATH)

obs, metric_step, reused = test(models, EUBO_fixed_radi, Data, Model_Params, Train_Params_Test)
(q_mu, _, q_z, _) = reused

In [ ]:
%time plot_samples(obs, q_mu, q_z, K, PATH)